# Modelo Gradient Boosting

Al igual que la anterior fase de limpieza e inspección, este notebook también se encontrará organizado en diferentes bloques, los cuáles son mostrados a continuación:

* **BLOQUE 1**: Partición de los conjuntos de datos
* **BLOQUE 2**: Entrenamiento del modelo con todas las variables
* **BLOQUE 3**: Reducción del número de variables
* **BLOQUE 4**: Ajuste de hiperparámetros y modelo final

Asimismo, las primeras celdas corresponderán a la instalación y carga de las librerías necesarias para la ejecución del script.

### Librerías

In [ ]:
#install.packages('xgboost')
#install.packages('caret')
#install.packages('e1071')
#install.packages('pROC')
#install.packages('Ckmeans.1d.dp')
#install.packages('ggplot2')
#install.packages('DiagrammeR')

library(xgboost)
library(caret)
library(e1071)
library(pROC)
library(Ckmeans.1d.dp)
library(ggplot2)
library(DiagrammeR)

## BLOQUE 1: Partición de los conjuntos de datos
En este primer bloque obtendremos los conjuntos necesarios para realizar el entrenamiento del modelo y poder observar y comparar el resultado del mismo.

In [ ]:
# Lectura del fichero .csv obtenido
data <- read.csv(???, sep = ???, header = ???)
head(data)

El objetivo de esta fase es la de dividir el dataset anterior en tres conjuntos distintos:

* Conjunto de entrenamiento
* Conjunto de validación
* Conjunto de test

In [ ]:
# Primera partición: Obtención del conjunto de test
n <- nrow(data)
set.seed(0)
train_index <- sample(???, size = ???)

entrenamiento <- data[train_index, ]  # Conjunto aún a dividir entre entrenamiento y validación
df_test <- data[-train_index, ]

In [ ]:
# Segunda partición: Obtención de los conjuntos de entrenamiento y validación
# IMPORTANTE!!: Balanceo de datasets. ¿Por qué?
cat('No defectos en el conjunto:', nrow(entrenamiento[entrenamiento$defecto == 0, ]), '\n')
cat('Defectos en conjunto:', nrow(entrenamiento[entrenamiento$defecto == 1, ]))

In [ ]:
# Segunda partición.

# Balanceo de conjuntos.
df_majority <- ???
df_minority <- ???

n_under <- nrow(df_minority)
n_over <- nrow(df_majority)

set.seed(0)
under_index <- sample(???, size = ???)

df_majority_undersampled <- df_majority[under_index, ]

# Concatenamos de nuevo los conjuntos balanceados.
entrenamiento_balanceado <- rbind(???, ???)

n2 <- nrow(entrenamiento_balanceado)
set.seed(0)
train_index2 <- sample(x = ???, size = ???)

df_train <- entrenamiento_balanceado[train_index2, ]
df_valid <- entrenamiento_balanceado[-train_index2, ]

In [ ]:
# Tamaño final de los conjuntos.
cat('Tamaño del conjunto de entrenamiento:', nrow(df_train), '\n')
cat('Tamaño del conjunto de validación:', nrow(df_valid), '\n')
cat('Tamaño del conjunto de test:', nrow(df_test))

In [ ]:
# Separamos de los conjuntos la variable objeto de estudio
# Convertir en matrices, ya que la función para entrenar el modelo XGB no permite utilizar dataframes
X_train <- as.matrix(df_train[, which(names(df_train) != ???)])
y_train <- df_train[, ???]

X_valid <- as.matrix(df_valid[, which(names(df_valid) != ???)])
y_valid <- df_valid[, ???]

X_test <- as.matrix(df_test[, which(names(df_test) != ???)])
y_test <- df_test[, ???]

## BLOQUE 2: Entrenamiento del modelo con todas las variables

#### **¿Qué es _Boosting_?**

_Boosting_ es un meta-algoritmo de aprendizaje automático que reduce el sesgo y la varianza en un contexto de aprendizaje supervisado. Consiste en combinar los resultados de varios clasificadores débiles para obtener un clasificador robusto. Cuando se añaden estos clasificadores débiles, se hace de modo que éstos tengan diferente peso en función de la exactitud de sus predicciones. Tras añadir un clasificador débil, los datos cambian su estructura de pesos: los casos mal clasificados ganan peso y los que son clasificados correctamente pierden peso. 

**Gradient Boosting (GB)** o _Potenciación del gradiente_ consiste en plantear el problema como una optimización numérica en el que el objetivo es minimizar una función de coste añadiendo clasificadores débiles mediante el descenso del gradiente. Involucra tres elementos:

* La **función de coste** a optimizar: depende del tipo de problema a resolver.
* Un **clasificador débil** para hacer las predicciones: por lo general se usan árboles de decisión.
* Un **modelo que añade (ensambla) los clasificadores débiles** para minimizar la función de coste: se usa el descenso del gradiente para minimizar el coste al añadir árboles.

Para este problema utilizaremos la librería _XGBoost_, que es una implementación particular muy eficiente de Gradient Boosting.

Tutoriales de la librería en R:
* https://xgboost.readthedocs.io/en/latest/R-package/xgboostPresentation.html
* http://dmlc.github.io/rstats/2016/03/10/xgboost.html

Los hiperparámetros más importantes que intervienen en este algoritmo y que aquí utilizaremos se describen a continuación:

* Parámetros generales:
 * **nthread**: número de hilos paralelos usados en la ejecución.
 * **objetive**: objetivo del aprendizaje.
 * **eval_metric**: métrica de evaluación para el conjunto en cuestión.
 
 
* Parámetros propios del _Boosting_:
 * **eta (learning rate)**: determina el impacto de cada árbol en la salida final. Se parte de una estimación inicial que se va actualizando con la salida de cada árbol. Es el parámetro que controla la magnitud de las actualizaciones.
 * **nrounds**: número de árboles a utilizar.


* Parámetros propios de los árboles:
 * **max_depth**: profundidad máxima de un árbol.
 
Más información sobre los parámetros y la librería en general:
* https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
params <- list(objetive = 'binary:logistic',
               nthread = 4,
               max_depth = 6,
               eta = 0.3,
               eval_metric = 'auc')

xgb_model <- xgboost(data = ???, label = ???, params = ???, nrounds = 30, verbose = 1, seed = 0)

In [ ]:
# Importancia de cada variable en el modelo
feature_importance <- xgb.importance(feature_names = ???, model = ???)
feature_importance

In [ ]:
# Gráfico con la importancia
options(repr.plot.width = 6, repr.plot.height = 3.5)
xgb.ggplot.importance(importance_matrix = feature_importance, rel_to_first = TRUE)

Para poder ver como de bueno es nuestro modelo, podemos obtener las predicciones que realiza sobre los conjuntos de entrenamiento y validación, y realizar el cálculo de alguna métrica para observar su rendimiento. En este caso, observaremos el **área bajo la curva ROC** (más conocido simplemente como AUC).

In [ ]:
# Ajuste/Rendimiento sobre datos de entrenamiento.
pred_train <- predict(xgb_model, ???)

# Rendimiento sobre datos de validación
pred_valid <- predict(xgb_model, ???)

In [ ]:
# Creamos un dataframe con la probabilidad de ser o no defectuosa la pieza
# y añadimos el cálculo de la predicción final

# Entrenamiento
predictions_train <- data.frame('probability' = pred_train, 
                                'prediction' = ???)

# Validación
predictions_valid <- data.frame('probability' = pred_valid, 
                                'prediction' = ???)

In [ ]:
# Cálculo de AUCs

# Entrenamiento
roc_train <- roc(???, predictions_train$prediction)
auc_train <- round(auc(roc_train), 4)

# Validación
roc_valid <- roc(???, predictions_valid$prediction)
auc_valid <- round(auc(roc_valid), 4)

In [ ]:
cat(paste('Área bajo la curva (AUC) en entrenamiento:', auc_train, '\n'))
cat(paste('Área bajo la curva (AUC) en validación:', auc_valid))

## BLOQUE 3: Reducción del número de variables

En este bloque se estudiará si es posible reducir el número de variables a utilizar por el _Gradient Boosting_. Se trata de una fase muy importante, pues, como ya se ha visto al medir la importancia, **no todas las variables causan un impacto importante** en el modelo. Además, utilizar un menor número de variables es preferible ya que implica una **simplificación el modelo** y una mayor interpretabilidad.

Para encontrar el número más óptimo de variables entrenaremos diversos modelos de manera iterativa, incorporando de una en una las variables ordenadas por importancia en el modelo. Así, almacenaremos en cada iteración los AUC tanto del conjunto de entrenamiento como de validación, y observaremos con que número de variables obtenemos un rendimiento lo suficientemente bueno.

In [ ]:
# Bucle for para obtener las AUCs del modelo según nº de variables
auc_features <- data.frame('num_variables' = numeric(), 'auc_train' = numeric(), 'auc_valid' = numeric())

for (i in 1:???){
    cat('Entrenamiento del modelo con ', i, ' variables.\n')
    cat('================================================')
    
    # Reducción de conjuntos
    cols <- c(feature_importance[1:i, '???'])$???
    X_train_red <- as.matrix(X_train[, cols])
    X_valid_red <- as.matrix(X_valid[, cols])
    
    # Ajuste del modelo y obtención de predicciones
    model_red <- xgboost(data = X_train_red, label = y_train, params = params, nrounds = 30,
                         verbose = 0, seed = 0)
    
    pred_train <- predict(model_red, X_train_red)
    pred_train <- ifelse(pred_train > 0.5, 1, 0)
    
    pred_valid <- predict(model_red, X_valid_red)
    pred_valid <- ifelse(pred_valid > 0.5, 1, 0)
    
    # AUCs
    roc_train <- roc(y_train, pred_train)
    auc_train <- round(auc(roc_train), 4)
    
    roc_valid <- roc(y_valid, pred_valid)
    auc_valid <- round(auc(roc_valid), 4)
    
    # Añadimos la información al dataframe
    auc_features[i, 'num_variables'] <- i
    auc_features[i, 'auc_train'] <- auc_train
    auc_features[i, 'auc_valid'] <- auc_valid
}

In [ ]:
# Dataframe que contiene la información del proceso iterativo
auc_features

In [ ]:
# Gráfico que permite visualizar el AUC de los conjuntos según el nº de variables utilizadas
options(repr.plot.width = 6, repr.plot.height = 3.5)

p <- ggplot(data = ???)
p <- p + geom_line(aes(x = ???, y = ???, colour = 'Entrenamiento'))
p <- p + geom_line(aes(x = ???, y = ???, colour = 'Validación'))
p <- p + theme_bw()
p <- p + theme(plot.title = element_text(size = 12, hjust = 0.5, face = "bold", color= "grey20"))
p <- p + labs(title = 'Gradient Boosting AUC según nº de variables')
p <- p + scale_x_continuous('Número de variables', breaks = seq(1, nrow(auc_features), 1))
p <- p + scale_y_continuous('AUC')
p <- p + scale_color_manual('', breaks = c('Entrenamiento', 'Validación'), 
                             values = c('Entrenamiento' = 'steelblue', 'Validación' = 'darkorange'))
p

In [ ]:
# Selección de las variables a utilizar
cols <- c(feature_importance[1:???, '???'])$???

# Reducción de los conjuntos
X_train_red <- X_train[, ???]
X_valid_red <- X_valid[, ???]
X_test_red <- X_test[, ???]

In [ ]:
# Nuevo modelo con la reducción de variables
xgb_model_red <- xgboost(data = ???, label = ???, params = ???, nrounds = 30,
                         verbose = 0, seed = 0)

In [ ]:
# Rendimiento del modelo en conjuntos de entrenamiento y validación (AUCs)

# Entrenamiento
pred_train <- predict(xgb_model_red, X_train_red)
pred_train <- ifelse(pred_train > 0.5, 1, 0)

roc_train <- roc(y_train, pred_train)
auc_train <- round(auc(roc_train), 4)

# Validación
pred_valid <- predict(xgb_model_red, X_valid_red)
pred_valid <- ifelse(pred_valid > 0.5, 1, 0)

roc_valid <- roc(y_valid, pred_valid)
auc_valid <- round(auc(roc_valid), 4)

# Resultados
cat('Área bajo la curva (AUC) en entrenamiento:', auc_train, '\n')
cat('Área bajo la curva (AUC) en validación:', auc_valid)

## BLOQUE 4: Ajuste de hiperparámetros y modelo final

Una vez seleccionadas las variables a utilizar en nuestro modelo, faltaría realizar el correspondiente **ajuste de los hiperparámetros**. Estos valores deber ser establecidos antes del entrenamiento, y de su correcta elección dependerá el resultado que obtendremos.

Los hiperparámetros ideales dependen del perfil de los datos que estamos analizando, por lo que no es sencillo establecer un procedimiento estándar para su obtención. En este caso realizaremos un **ajuste manual** de los mismos, observando directamente el cambio en el rendimiento del modelo aplicado a los datos de validación.

In [ ]:
# Cambio en los valores de los hiperparámetros
best_params <- list(objetive = 'binary:logistic',
                    nthread = 4,
                    max_depth = ???,
                    eta = ???,
                    eval_metric = 'auc')

# Entrenamiento del modelo con los nuevos valores
best_xgb_model <- xgboost(data = X_train_red, label = y_train, params = best_params, nrounds = ???,
                          verbose = 0, seed = 0)

# Rendimiento del nuevo modelo en los conjuntos de entrenamiento y validación
# Entrenamiento
new_pred_train <- predict(best_xgb_model, X_train_red)
new_pred_train <- ifelse(new_pred_train > 0.5, 1, 0)

new_roc_train <- roc(y_train, new_pred_train)
new_auc_train <- round(auc(new_roc_train), 4)

# Validación
new_pred_valid <- predict(best_xgb_model, X_valid_red)
new_pred_valid <- ifelse(new_pred_valid > 0.5, 1, 0)

new_roc_valid <- roc(y_valid, new_pred_valid)
new_auc_valid <- round(auc(new_roc_valid), 4)

# Resultados
cat('Nuevo área bajo la curva (AUC) en entrenamiento:', new_auc_train, '\n')
cat('Nuevo área bajo la curva (AUC) en validación:', new_auc_valid)

Obtenido ya nuestro mejor modelo, podemos observar su rendimiento sobre unos datos totalmente nuevos y más reales: el conjunto de test. Para ello, además de mostrar el AUC como veníamos haciendo a lo largo del análisis, observaremos también la **matriz de confusión**:

In [ ]:
# Predicciones en conjunto de test
pred_test <- predict(best_xgb_model, ???)
pred_test <- ???

# Área bajo la curva
roc_test <- roc(???, pred_test)
auc_test <- round(auc(roc_test), 4)

cat('Área bajo la curva (AUC) en test:', auc_test)

In [ ]:
# Matriz de confusión
cm_test <- ???

cat('Matriz de confusión para datos de test:\n\n')
print(cm_test)

In [ ]:
# Otra manera de obtener la matriz de confusión, junto con otras métricas
cm_test2 <- confusionMatrix(as.factor(???), as.factor(???), mode = 'everything',
                            positive = '1')
cm_test2

También es posible si se desea observar los árboles que son construidos en el modelo y que establecen las relaciones entre las variables utilizadas en cada uno de ellos:

In [ ]:
# IMPORTANTE!!: Tiempo de ejecución muy largo. Evitar ejecutar
xgb.plot.tree(feature_names = best_xgb_model$feature_names, model = best_xgb_model, trees = 0)

Para terminar, debemos guardar nuestro modelo ya entrenado, de manera que cuando se desee utilizar solo sea necesario cargarlo y aplicarlo a nuevos conjuntos de datos.

In [ ]:
# Guardado del modelo ya entrenado
xgb.save(???, ???)